# Feature Engineering 1.0

As I started playing w/ logistic regression to build a Lada Gaga song detector, I found that the # of features is too large to compute.  Moreso, the feature grid was mostly sparse and mostly useless since it was based on a basic word count vectorization.  

There is some good scikit info on managing this:
http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html#sphx-glr-auto-examples-text-document-classification-20newsgroups-py  

This is a survey of my findings


